In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Armado del dataset

#### Votos validos

In [ ]:
df = pd.read_csv('presidente_afirmativos.csv')

In [ ]:
df.shape

In [ ]:
df.columns

#### Mesas

In [ ]:
mesas_df = df[['distrito_nombre', 'seccion_nombre', 'circuito_nombre', 'mesa_id', 'votos_cantidad']].groupby([
    'distrito_nombre', 'seccion_nombre', 'circuito_nombre', 'mesa_id'
]).sum().reset_index()

In [ ]:
mesas_df.rename(columns = {'votos_cantidad': 'votos_mesa_total'}, inplace=True)

In [ ]:
mesas_df = mesas_df[mesas_df['votos_mesa_total'] > 0]

In [ ]:
df = pd.merge(df, mesas_df)

In [ ]:
df['percentage'] = df['votos_cantidad'] / df['votos_mesa_total']

#### Ganadores por mesa

In [ ]:
ganadores_por_mesa = df.sort_values('votos_cantidad', ascending=False).groupby(
    ['distrito_nombre', 'seccion_nombre', 'circuito_nombre', 'mesa_id'
]).first().reset_index()

#### Votos invalidos

In [ ]:
invalidos = pd.read_csv('presidente_invalidos.csv')

In [ ]:
invalidos.shape

In [ ]:
invalidos['votos_tipo'].value_counts()

In [ ]:
invalidos.head()

In [ ]:
mesas_invalidos = invalidos.pivot(columns=['votos_tipo'], 
                index=[ 'distrito_nombre',
                        'seccion_nombre',
                        'circuito_nombre',
                        'mesa_id'],
               values='votos_cantidad').reset_index()

In [ ]:
mesas_invalidos.head()

In [ ]:
total_mesas = pd.merge(mesas_df, mesas_invalidos)

In [ ]:
total_mesas['total_invalidos'] = total_mesas[
    ['COMANDO', 'IMPUGNADO', 'NULO', 'RECURRIDO']].sum(axis=1)

In [ ]:
total_mesas['total_no_afirmativos'] = total_mesas[
    ['COMANDO', 'EN BLANCO', 'IMPUGNADO', 'NULO', 'RECURRIDO']].sum(axis=1)

In [ ]:
main_df = pd.merge(total_mesas, ganadores_por_mesa)

In [ ]:
main_df['invalidos_percentage'] = main_df['total_invalidos'] / main_df['votos_mesa_total']

#### Votos por agrupacion por mesa

In [ ]:
votos_por_mesa = df.pivot(columns=['agrupacion_nombre'], 
                index=[ 'distrito_nombre',
                        'seccion_nombre',
                        'circuito_nombre',
                        'mesa_id'],
               values='votos_cantidad').reset_index()

In [ ]:
votos_por_mesa = pd.merge(votos_por_mesa, total_mesas)

## Distribucion de mesas por % sacado

In [ ]:
for a, a_df in df.groupby('agrupacion_nombre'):
    fig = plt.figure(figsize=(15, 8))
    plt.title(a)
    a_df['percentage'].hist(bins=100)

## Mesas con % de votos de un partido

In [ ]:
set_percentage = 0.9

In [ ]:
mesas_percentage = ganadores_por_mesa[ganadores_por_mesa['percentage'] >= set_percentage]

#### Cantidad de mesas por agrupacion

In [ ]:
mesas_percentage['agrupacion_nombre'].value_counts()

In [ ]:
mesas_percentage[mesas_percentage['agrupacion_nombre']=='UNION POR LA PATRIA'].sort_values(
    'percentage', ascending=False)

#### Cantidad de mesas por agrupacion y distrito

In [ ]:
mesas_percentage[['agrupacion_nombre', 'distrito_nombre', 'votos_cantidad']].groupby(
    ['agrupacion_nombre', 'distrito_nombre']).count().sort_values(
    ['agrupacion_nombre', 'votos_cantidad'], ascending=False)

#### Cantidad de votos por agrupacion

In [ ]:
mesas_percentage[['agrupacion_nombre', 'votos_cantidad']].groupby(
    'agrupacion_nombre').sum().sort_values('votos_cantidad', ascending=False)

#### Cantidad de mesas por agrupacion y distrito

In [ ]:
mesas_percentage[['agrupacion_nombre', 'distrito_nombre', 'votos_cantidad']].groupby(
    ['agrupacion_nombre', 'distrito_nombre']).sum().sort_values(
    ['agrupacion_nombre', 'votos_cantidad'], ascending=False)

## Porcentaje de victoria

#### Total

In [ ]:
percentage_dfs = []

In [ ]:
for p, p_df in ganadores_por_mesa.groupby('agrupacion_nombre'):
    temp_df = p_df['percentage'].describe().to_frame()
    temp_df.columns = [p]
    percentage_dfs.append(temp_df.T)

In [ ]:
pd.concat(percentage_dfs)

## Votos no validos

#### % de mesas sin invalidos por agrupacion ganadora

In [ ]:
zero_invalidos = main_df[main_df['total_invalidos'] == 0].copy()

In [ ]:
agrupacion_grp_sin = zero_invalidos['agrupacion_nombre'].value_counts().to_frame().reset_index()

In [ ]:
agrupacion_grp_sin.columns = ['agrupacion_nombre', 'sin_invalidos']

In [ ]:
agrupacion_grp_total = main_df['agrupacion_nombre'].value_counts().to_frame().reset_index()

In [ ]:
agrupacion_grp_total.columns = ['agrupacion_nombre', 'total']

In [ ]:
agrupaciones_ganadoras_invalidos = pd.merge(agrupacion_grp_sin, agrupacion_grp_total)

In [ ]:
agrupaciones_ganadoras_invalidos['percentage'] = agrupaciones_ganadoras_invalidos[
    'sin_invalidos'] / agrupaciones_ganadoras_invalidos['total']

In [ ]:
agrupaciones_ganadoras_invalidos.sort_values('percentage', ascending=False)

#### Mesas ganadores con mas invalidos por agrupacion

In [ ]:
dfs = []

In [ ]:
for p, p_df in main_df.groupby('agrupacion_nombre'):
    temp_df = p_df['invalidos_percentage'].describe().to_frame()
    temp_df.columns = [p]
    dfs.append(temp_df.T)

In [ ]:
inv_dist_df = pd.concat(dfs)

In [ ]:
inv_dist_df

In [ ]:
for v in ['mean', '25%', '50%', '75%', 'max']:
    fig = plt.figure(figsize=(15, 8))
    plt.title(v)
    plot_df = inv_dist_df.sort_values(v)
    x_values = plot_df.index
    plt.bar(x_values, plot_df[v])

## Mesas con 0 votos

In [ ]:
partido = 'LA LIBERTAD AVANZA'

In [ ]:
zero_votos = votos_por_mesa[votos_por_mesa[partido] == 0]

In [ ]:
zero_votos['total_invalidos'].value_counts()

In [ ]:
zero_votos[zero_votos['total_invalidos']==96]